# Initial Database EDA
----



### Lighthouse Labs, Midterm Project Project - Predicting Flight Delays.

##### January 13, 2023. Terre Leung, Tetiana Fesenko, and Jamie Dormaar

---


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime as dt

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load data tables:
# flights_initial_500000_records    = pd.read_csv('../data/flights_initial_500000_records.csv', delimiter= ',')
# flights_delay_dates_all_records   = pd.read_csv('../data/flights_delay_dates_all_records.csv', delimiter= ',')
flights_random_100000_records     = pd.read_csv('../data/flights_random_100000_records.csv', delimiter= ',')
# flights_random_5000_records       = pd.read_csv('../data/flights_random_5000_records.csv', delimiter= ',')
flights_test_all_records          = pd.read_csv('../data/flights_test_all_records.csv', delimiter= ',')
# fuel_consumption_all_records      = pd.read_csv('../data/fuel_consumption_all_records.csv', delimiter= ',')
# passengers_initial_300000_records = pd.read_csv('../data/passengers_initial_300000_records.csv', delimiter= ',')
# flights_usa = pd.read_csv('../data/usa_flights2.csv', delimiter= ',')  # Terre is there a new csv to go with this one?

In [ ]:
# Save working copies of the data:
# df_fl_init    = flights_initial_500000_records.copy()
# df_fl_delays  = flights_delay_dates_all_records.copy()
df_fl_smpl1   = flights_random_100000_records.copy()
# df_fl_smpl2   = flights_random_5000_records.copy()
df_fl_test    = flights_test_all_records.copy()
# df_fc         = fuel_consumption_all_records.copy()
# df_pa_init    = passengers_initial_300000_records.copy()

##### SETUP:

In [ ]:
df = df_fl_smpl1.copy()

In [ ]:
# # Session stamp
# tag = 'EDA1_' 
# dt = ''
# session = f'{tag}{dt}'

##### LOAD TABLE:

In [ ]:
# flights
print(f'\nflights.shape: {df.shape}')
display(df.head(3))

##### NOTE: Missing Data.


In [ ]:
# Check for nulls:
# flights Table percent Null content:
df_nulls = df.isnull().sum().sort_values(ascending= False)
perc = (df.isnull().sum()/df.isnull().count()).sort_values(ascending = False)
df_missing_data = pd.concat(
    [df_nulls, perc]
  , axis=1
  , keys=['Total', 'Percent']
  , verify_integrity= True
)
print(f'\nflights_missing_data.head(20)')
display(df_missing_data.head(20))

##### NOTE: Differences between flights, and flights_test table data:

In [ ]:
fl_test_exclusion = df[df.columns[~df.columns.isin(df_fl_test.columns)]]
fl_test_exclusion.head(3)

In [ ]:
flights_columns = list(df.columns)
flights_test_columns = list(df_fl_test.columns)
fl_test_exclusion_columns = list(fl_test_exclusion.columns)
unique_columns = list(set(flights_columns + flights_test_columns))

In [ ]:
print(len(flights_columns))
print(len(flights_test_columns))
print(len(unique_columns))

### DROP: rows with Null arr_delay

In [ ]:
print(df.shape)
df.dropna(subset= ['arr_delay'], inplace= True)
print(df.shape)

#### Isolate the columns that will be great predictors but will not be available 1 week before departure:


In [ ]:
day_of_delays = [
    'carrier_delay',
    'weather_delay',
    'nas_delay',
    'security_delay',
    'late_aircraft_delay',
    'dep_delay'
]

# 'taxi_in',                ???
# 'taxi_out',               ???
# 'wheels_on',              ???
# 'wheels_off',             ???
# 'total_add_gtime',        ???
# 'longest_add_gtime',      ???
# 'actual_elapsed_time',    ???

### DROP: columns not available to predict - 1 week.


In [ ]:
print(df.shape)
df = df.drop(columns=day_of_delays)
print(df.shape)

#### Starting with only the columns that are in the test table:


In [ ]:
df[[
    'origin',
    'dest',
    'origin_airport_id',
    'dest_airport_id',
    'origin_city_name',
    'dest_city_name',
]].head()

- Airport code and ids are redundant, can drop one.  
- City names are only useful if we want to extract the state codes for the models:

In [ ]:
df[[
    'flights',
    'mkt_carrier',
    'mkt_carrier_fl_num',
    'mkt_unique_carrier',
    'op_carrier_fl_num',
    'op_unique_carrier',
    'tail_num'
]].head()

In [ ]:
# Are the carrier codes ever different?
carrier = df[['mkt_carrier','mkt_unique_carrier','op_unique_carrier']].copy()
carrier['mkt_carrier'].nunique()

In [ ]:
carrier.groupby(['mkt_carrier','mkt_unique_carrier'], as_index=False).count().sort_values('op_unique_carrier')

- mkt_carrier and mkt_unique_carrier appear to be duplicates. One can be dropped.
- maybe keep to encode and see if there are delay related trends?

In [ ]:
# Is there ever more than one value under 'flights'?
df['flights'].nunique()

- 'flights' can be dropped, as it offers no unique info

In [ ]:
# Do carrier flight numbers vary? and are they redundant with tail_num?
plane_nums = df[[
    'mkt_carrier_fl_num',
    'op_carrier_fl_num',
    'tail_num'
]].copy()
plane_nums.groupby(
    ['mkt_carrier_fl_num', 'op_carrier_fl_num']
    , as_index=False).count().sort_values('op_carrier_fl_num')

In [ ]:
mkt_diff = plane_nums[plane_nums['mkt_carrier_fl_num']!=plane_nums['op_carrier_fl_num']]
print(mkt_diff.shape)
mkt_diff

12 rows of 100,000 not enough difference to care about.  
- Drop one permanently, and 
- shuffle other two to later if we have time for advanced Feature Engineering.

In [ ]:
df[[
    'crs_arr_time',
    'crs_dep_time',
    'crs_elapsed_time',
]].head()

- These all appear unique and are numeric, so they're good to stay for now.

In [ ]:
df['branded_code_share'].value_counts()

In [ ]:
drop_columns = [
      'origin_airport_id'
    , 'dest_airport_id'
    , 'mkt_carrier'
    , 'op_unique_carrier'
    , 'branded_code_share'
    , 'mkt_carrier_fl_num'
    , 'tail_num'
    , 'origin_city_name'
    , 'dest_city_name'
    , 'flights'
    , 'op_carrier_fl_num'
    , 'mkt_unique_carrier'
]

### DROP: columns either redundant, or without unique data.

In [ ]:
print(df.shape)
df = df[df.columns[~df.columns.isin(drop_columns)]]
print(df.shape)

In [ ]:
df[[
'wheels_on',
'wheels_off',
'taxi_in',
'taxi_out'
]].head()

These are fine, but I wonder if wheels_on, wheels_off, taxi_in, and taxi_out are day-of items...

In [ ]:
df['first_dep_time'].value_counts().head()

In [ ]:
df[[
    'air_time'
  , 'first_dep_time'
  , 'dep_time'
  , 'arr_time'
]].head()

In [ ]:
unlikely = [
    'air_time'
  , 'first_dep_time'
  , 'dep_time'
]

prob_also_day_of = [
    'taxi_in'
  , 'taxi_out'
  , 'wheels_on'
  , 'wheels_off'
  , 'total_add_gtime'
  , 'longest_add_gtime'
  , 'actual_elapsed_time'
]

drop_columns = [
    'dup'
  , 'diverted'
  , 'cancelled'
  , 'cancellation_code'
  , 'no_name'
]

### DROP: columns

In [ ]:
print(df.shape)
df = df[df.columns[~df.columns.isin(
      prob_also_day_of + drop_columns
      )]]
print(df.shape)

### Summary:

- 'crs_arr_time',
- 'crs_dep_time',
- 'crs_elapsed_time',
- 'origin',
- 'dest',
- 'distance',
- 'fl_date',


##### drop for now:
Airport ids
- 'origin_airport_id',
- 'dest_airport_id',

Carrier codes:
- 'mkt_carrier', (&/or 'mkt_unique_carrier'),
- 'op_unique_carrier',
- 'branded_code_share',

Flight mkt_carrier and tail nums:
- 'mkt_carrier_fl_num', (&/or 'op_carrier_fl_num'),
- 'tail_num',

Possible feature engineering later:
- 'origin_city_name',
- 'dest_city_name',

Maybe dropped forever:
- 'flights',
- 'op_carrier_fl_num',
- 'mkt_unique_carrier'

In [ ]:
df.head()

### NUMERIC ORDINAL ENCODING:

In [ ]:
keys = set(list(df['origin']) + list(df['dest']))
values = len(keys)
airport_code_map = dict(zip(keys, range(values)))
# airport_code_map

In [ ]:
df['origin'] = df['origin'].map(airport_code_map)
df['dest'] = df['dest'].map(airport_code_map)

In [ ]:
df.head()

### OUTLIERS: 1.5 * IQR

In [ ]:
# Instantiate the Arrival Delays:
delays = df['arr_delay']

# Define the quantiles of the delay distribution:
Q1 = delays.quantile(0.25)
Q3 = delays.quantile(0.75)
IQR = Q3 - Q1

# Define the outlier thresholds
min_threshold = (Q1 - 1.5 * IQR)
max_threshold = (Q3 + 1.5 * IQR)

df = df[~((delays < min_threshold)|(delays > max_threshold))]
df.shape

### DATETIME FEATURES:

In [ ]:
# Convert fl_date from string to datetime data type
df[['fl_date']] = df[['fl_date']].apply(pd.to_datetime)

df['year'] = df['fl_date'].dt.year
df['month'] = df['fl_date'].dt.month
df['day_of_wk'] = df['fl_date'].dt.dayofweek

In [ ]:
df.head()

### SAVE CLEANED TABLE:

In [ ]:
df.to_csv('../data/df_numeric.csv', index=False)